In [1]:
import sys
import os
current_dir = os.getcwd()
ds_path = os.path.join(os.path.dirname(current_dir), 'ds')
sys.path.append(ds_path)

from generate_loader import dataloaders

In [2]:
trainloader, testloader = dataloaders()